In [3]:
# Import the pandas package, then use the "read_csv" function to read
# the labeled training data
import pandas as pd       
train = pd.read_csv("insult/train.csv", header=0)

In [4]:
print(train.shape)
print(train.columns.values)
print(train.head())
print(train.describe())

(3947, 3)
['Insult' 'Date' 'Comment']
   Insult             Date                                            Comment
0       1  20120618192155Z                               "You fuck your dad."
1       0  20120528192215Z  "i really don't understand your point.\xa0 It ...
2       0              NaN  "A\\xc2\\xa0majority of Canadians can and has ...
3       0              NaN  "listen if you dont wanna get married to a man...
4       0  20120619094753Z  "C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd...
            Insult
count  3947.000000
mean      0.265771
std       0.441799
min       0.000000
25%       0.000000
50%       0.000000
75%       1.000000
max       1.000000


In [5]:
print(train.Comment[0])

"You fuck your dad."


In [8]:
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords # Import the stop word list
def comment_to_words( raw_comment ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    comment_text = BeautifulSoup(raw_comment).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", comment_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))

In [9]:
clean_comment = comment_to_words( train["Comment"][0] )
print clean_comment

fuck dad


/Users/mohitsharma/anaconda/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /Users/mohitsharma/anaconda/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [10]:
num_comments = train["Comment"].size
print(num_comments)
clean_train_comments = []
for i in xrange( 0, num_comments ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
     # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Comment %d of %d\n" % ( i+1, num_comments ) 
    clean_train_comments.append( comment_to_words( train["Comment"][i] ) )

3947
Comment 1000 of 3947

Comment 2000 of 3947

Comment 3000 of 3947



In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 
train_data_features = vectorizer.fit_transform(clean_train_comments)
train_data_features = train_data_features.toarray()

In [12]:
print(train_data_features.shape)

(3947, 5000)


In [13]:
vocab = vectorizer.get_feature_names()
print(vocab)

[u'aamir', u'aap', u'ab', u'abandoned', u'abe', u'ability', u'able', u'abortion', u'abortions', u'abroad', u'abs', u'abscam', u'absolute', u'absolutely', u'abuse', u'abused', u'abusing', u'accept', u'accepted', u'accepting', u'access', u'accident', u'according', u'account', u'accountable', u'accurate', u'accused', u'achieve', u'achievements', u'across', u'act', u'acting', u'action', u'actions', u'actively', u'activists', u'activities', u'actor', u'acts', u'actual', u'actually', u'ad', u'adam', u'add', u'added', u'addition', u'additional', u'address', u'adds', u'administration', u'admire', u'admit', u'admitted', u'admittedly', u'adopted', u'adult', u'adults', u'advantage', u'advice', u'advocate', u'affair', u'affairs', u'affect', u'affluent', u'afford', u'afghanistan', u'afraid', u'africa', u'african', u'ag', u'age', u'agencies', u'agenda', u'agent', u'ages', u'ago', u'agree', u'agreed', u'ah', u'ahead', u'aint', u'air', u'ak', u'aka', u'al', u'alabama', u'album', u'albums', u'alcohol',

In [14]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit( train_data_features, train["Insult"] )

In [15]:
import pandas as pd       
test = pd.read_csv("insult/test.csv", header=0)

In [16]:
num_comments = test["Comment"].size
print(num_comments)
clean_test_comments = []
for i in xrange( 0, num_comments ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
     # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Comment %d of %d\n" % ( i+1, num_comments ) 
    clean_test_comments.append( comment_to_words( test["Comment"][i] ) )

2235
Comment 1000 of 2235

Comment 2000 of 2235



In [17]:
test_data_features = vectorizer.transform(clean_test_comments)
test_data_features = test_data_features.toarray()

In [19]:
result = forest.predict(test_data_features)
output = pd.DataFrame( data={"Insult":result,"Date":test["Date"], "Comment":test["Comment"]} )
output.to_csv( "social_comment_insult.csv", index=False, quoting=3,escapechar='\\' )